# Covid-19 na cidade de Viçosa-MG

Analise da pandemia de covid-19 doença causada pelo novo coronavírus na cidade de Viçosa-MG.
Todos os dados foram extraídos dos boletins epidemiologicos divulgados pela prefeitura em seu grupo no telegram.

In [1]:
import pandas as pd
from kaleido.scopes.plotly import PlotlyScope
from sqlalchemy import create_engine
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from collections import Counter, defaultdict
import matplotlib.pyplot as plt
import os

scopes = PlotlyScope()
pio.templates.default = 'simple_white'

#with open('somethin.png', 'wb') as f:
#    f.write(scopes.transform(fig, format='png')

"\nwith open('somethin.png', 'wb') as f:\n    f.write(scopes.transform(fig, format='png')\n"

In [2]:
engine = create_engine(os.getenv('DB_URL'))

In [3]:
df = pd.read_sql_table('casos', engine, index_col='id')

df.head()

,sexo,idade,data,semana_epidemiologica,sintomatico,agente_saude
id,,,,,,
3,None,NaN,2020-04-23,1,True,False
47,M,51.0,2020-06-18,9,True,False
46,M,42.0,2020-06-18,9,True,False
45,F,26.0,2020-06-16,9,True,False
44,F,36.0,2020-06-16,9,True,False


os valores faltantes foi porque não foram divulgados no grupo do telegram, podendo ter sido divulgados em outro canal, sendo que esses outros canais não foram consultados para rever todos os casos apenas uma ou outra vez.

Dicionario de variável:
- sexo: o sexo da pessoa que foi contaminada, F é feminino, e M é masculino;  
- idade: a idade da pessoa contaminada, para pessoas com menos do que 1 ano, o valor foi arrendondado;  
- data: a data na qual foi divulgado o caso;  
- semana_epidemiologica: semana na qual foi divulgado o caso, começando a contar desde o primeiro boletim liberado;  
- sintomatico: Se a pessoa teve ou não sintomas da doença, este valor quando não informado pela prefeitura esta como true, por esse ser o valor mais comum.

Aqui vemos o gráfico de distribuição de casos por sexo.

In [4]:
dfsex = df['sexo'].dropna()

px.histogram(dfsex, x='sexo', nbins=3, color='sexo').update_layout(yaxis_title='Quantidade', title_text='Contagio por sexo')

Neste segundo gráfico vemos a distribuição de sintomaticos e assintomaticos por sexo.

In [5]:
df['sintomatico'] = df['sintomatico'].map({True: 'Sintomático', False: 'Assintomático'})
px.histogram(df, x='sexo', nbins=3, color='sintomatico').update_layout(yaxis_title='Quantidade', title_text='Quantidade assintomático por sexo')

Aqui vemos os casos por semana.

In [6]:
daily_cases = defaultdict(int)

data = df['data']
age = df['idade']
data.dropna(inplace=True)
age.dropna(inplace=True)

age_cases = Counter(age)

for i in data:
    daily_cases[i.date().strftime('%d/%m/%Y')] += 1

In [7]:
daily = []
agely = []

for i in daily_cases.keys():
    daily.append((i, daily_cases.get(i)))
for i in age_cases.keys():
    agely.append((i, age_cases.get(i)))

import datetime

daily.sort(key=lambda x: datetime.datetime.strptime(x[0], '%d/%m/%Y'))

daily_cases = pd.DataFrame(daily)
age_cases = pd.DataFrame(agely)
daily_cases.columns = ['Data', 'Quantidade']
age_cases.columns = ['Idade', 'Quantidade']

Ja neste gráfico vemos a quantidade de casos por dia.* Os dias que não aparecem no gráfico é porque não tiveram nenhum caso confirmado no mesmo*.

In [34]:
go.Figure().add_trace(go.Scatter(x=daily_cases['Data'], y=daily_cases['Quantidade'], mode='lines')).update_layout(xaxis_title='Data', yaxis_title='Quantidade', title_text='casos por dia')

Neste gráfico temos a distruibuição de casos por idade. Usando o auxilio visual, percebemos que quanto mais quente a cor, mas casos teve naquela idade, sendo assim o oposto também vale, quanto mais fria a cor, menos casos.

In [9]:
px.bar(data_frame=age_cases, x='Idade', y='Quantidade', width=800, height=600).update_layout(title_text='Casos por idade')

In [10]:
new_df = pd.read_sql_table('quantidade_dia', engine, index_col='id')

new_df.sort_values(by='data', inplace=True)

new_df

,casos,recuperados,data,semana,mortes
id,,,,,
2,0,0,2020-04-19,1,0
3,0,0,2020-04-20,1,0
4,0,0,2020-04-21,1,0
5,0,0,2020-04-22,1,0
6,3,0,2020-04-23,1,0
...,...,...,...,...,...
171,7,5,2020-10-05,25,0
172,1,4,2020-10-06,25,0
173,6,7,2020-10-07,25,0


Aqui os dados faltantes segue o mesmo raciocinio do DataFrame anterior.

dicionario de variáveis:
- casos: quantidade de casos confirmados naquele dia;
- recuperados: quantidade de pessoas que se recuperaram da covid-19 na data em questão;
- data: a data da qual os dados fazem alusão;
- semana: a semana epidemiologica na qual aquela data e dados fazem parte, seguindo o mesmo raciocinio do DataFrame anterior.

Neste gráfico, temos a distribuição dos novos casos, e casos recuperados por dia dos 7 últimos dias.

In [11]:
px.bar(new_df.iloc[-7:], x='data', y=['casos', 'recuperados', 'mortes'], color_discrete_sequence=['#7E2EA6', '#1FA663', '#A6833F'], width=800, height=600, labels={'variable': 'Tipo de caso'}).update_layout(barmode='group', title_text='Recuperados x Mortos x Contaminados por dia')

Gráfico mostrando progressão de casos por dia

In [35]:
go.Figure().add_trace(go.Scatter(x=new_df['data'], y=new_df['casos'], mode='lines', marker_color='#333')).update_layout(xaxis_title='Data', yaxis_title='Quantidade', title_text='Contaminados por dia')

In [36]:
go.Figure().add_trace(go.Scatter(x=new_df['data'], y=round(new_df['casos'].rolling(7).mean(), 2), mode='lines', marker_color='#633')).update_layout(xaxis_title='Data', yaxis_title='Media movel', title_text='Média movel de casos por dia')

Gráfico com a tendência de casos recuperados por dia.

In [37]:
go.Figure().add_trace(go.Scatter(x=new_df['data'], y=new_df['recuperados'], mode='lines', marker_color='blue')).update_layout(xaxis_title='Data', yaxis_title='Quantidade', title_text='Recuperados por dia')

In [38]:
go.Figure().add_trace(go.Scatter(x=new_df['data'], y=round(new_df['recuperados'].rolling(7).mean(), 2), mode='lines', marker_color='#0D0C9C')).update_layout(xaxis_title='Data', yaxis_title='Media movel', title_text='Média movel de recuperados por dia')

In [39]:
go.Figure().add_trace(go.Scatter(x=new_df['data'], y=new_df['mortes'], mode='lines', marker_color='purple')).update_layout(xaxis_title='Data', yaxis_title='Quantidade', title_text='Mortes por dia')

In [40]:
go.Figure().add_trace(go.Scatter(x=new_df['data'], y=round(new_df['mortes'].rolling(7).mean(), 2), mode='lines', marker_color='#DD1169')).update_layout(xaxis_title='Data', yaxis_title='Media movel', title_text='Média movel de recuperados por dia')

Aqui criamos um novo DataFrame com o total de casos e total de casos recuperados em cada data.

In [18]:
total_daily = []

total_casos = 0
total_recuperados = 0
total_mortes = 0

for (data, casos, recuperados, mortes) in zip(new_df['data'], new_df['casos'], new_df['recuperados'], new_df['mortes']):
    total_casos += casos
    total_recuperados += recuperados
    total_mortes += mortes
    total_daily.append((data.date().strftime('%d/%m/%Y'), total_casos, total_recuperados, total_mortes))


total = pd.DataFrame(total_daily)
total.columns = ['Data', 'Total casos', 'Total recuperados', 'Total mortes']

total.tail()

,Data,Total casos,Total recuperados,Total mortes
169,05/10/2020,409,357,3
170,06/10/2020,410,361,3
171,07/10/2020,416,368,3
172,08/10/2020,425,372,3
173,09/10/2020,429,393,3


Gráfico mostrando o total de casos e total de casos recuperados por data, gráfico com o número total de casos na escala log, podemos notar, que de acordo com os dados oficiais da prefeitura, houveram 2 vezes na qual o número de casos e de recuperados eram iguais sendo essas vezes nas datas de 30/04/20 - 05/05/20 e de 13/05/20 - 18/05/20

In [19]:
fig = go.Figure(data=[
    go.Scatter(name='Contaminados', x=total['Data'], y=total['Total casos'], marker_color='#7E2EA6', mode='lines+markers'),
    go.Scatter(name='Recuperados', x=total['Data'], y=total['Total recuperados'], marker_color='#1FA663', mode='lines+markers'),
    go.Scatter(name='Recuperados', x=total['Data'], y=total['Total mortes'], marker_color='#A6833F', mode='lines+markers')
]).update_layout(width=800, height=600, title_text='Total casos X Total recuperados X Total mortes', yaxis_type='log')
with open('graph1.1.png', 'wb') as f:
    f.write(scopes.transform(fig, format='png'))

Criamos mais um DataFrame, este contendo a quantidade de casos ativos por data

In [20]:
total_ativo = []

total_casos_ativos = 0

for (data, casos, recuperados, mortes) in zip(new_df['data'], new_df['casos'], new_df['recuperados'], new_df['mortes']):
    total_casos_ativos += casos - recuperados - mortes
    total_ativo.append((data, total_casos_ativos))

total_ativo = pd.DataFrame(total_ativo)
total_ativo.columns = ['Data', 'Casos Ativos']
total_ativo.tail()

,Data,Casos Ativos
169,2020-10-05,49
170,2020-10-06,46
171,2020-10-07,45
172,2020-10-08,50
173,2020-10-09,33


Neste gráfico vemos a tendência de casos ativos, e também a quantidade dos mesmos por dia, podemos notar que esta em uma tendência de crescimento, havendo muitas variações, mas está crescendo. Os primeiros pontos que estão como não tendo nenhum caso ativo, foi antes de ter sido notificados quaisquer casos na cidade, sendo esses dados começando no dia 19/04/20, que é o primeiro dia da primeira semana na qual começaram os boletins epidemiologicos.

In [41]:
go.Figure(data=[
    go.Scatter(name='Casos ativos', x=total_ativo['Data'], y=total_ativo['Casos Ativos'], mode='lines')
]).update_layout(width=800, height=600, title_text='Casos ativos')

In [42]:
go.Figure().add_trace(go.Scatter(x=total_ativo['Data'], y=round(total_ativo['Casos Ativos'].rolling(7).mean(), 2), mode='lines', marker_color='#C0D')).update_layout(xaxis_title='Data', yaxis_title='Media movel', title_text='Média movel de casos ativos por dia')

Aqui vemos a porcentagem de casos asintomaticos x sintomaticos, podemos notar que a quantidade de casos sintomaticos é bem maior do que a de casos asintomaticos, mas como já citado anteriormente, os dados faltantes dessa categoria foram entendidos como sendo sintomaticos, por ser muito mais comum.

In [23]:
px.histogram(data_frame=df, x='sintomatico', histnorm='percent').update_layout(yaxis_title='Porcentagem', title_text='Porcentagem de Sintomáticos x Asintomáticos')

Neste gráfico vemos a distribuição da idade dos contaminados por sexo

In [24]:
df_new = df.copy()

df_new.dropna(inplace=True)

px.histogram(data_frame=df_new, x='idade', color='sexo',color_discrete_sequence=['#219C43', '#E85F5F'], title='Idade por sexo').update_layout(xaxis_title='Idade', yaxis_title='Quantidade')

In [25]:
from apyori import apriori
res = list(engine.execute("SELECT string_agg(s.sintoma, ',') FROM caso_sintoma cs JOIN sintomas s on s.id = cs.sintoma_id GROUP BY cs.caso_id").fetchall())
agg = [i[0].split(',') for i in res]
[(x.items) for x in filter(lambda x: len(x.items) > 1, apriori(agg, min_lenght=2))]

[frozenset({'Coriza', 'Dor de cabeça'}),
 frozenset({'Coriza', 'Tosse'}),
 frozenset({'Dor de cabeça', 'Dor muscular'}),
 frozenset({'Dor de cabeça', 'Febre'}),
 frozenset({'Dor de cabeça', 'Perda Olfato'}),
 frozenset({'Dor de cabeça', 'Perda paladar'}),
 frozenset({'Dor de cabeça', 'Tosse'}),
 frozenset({'Febre', 'Tosse'}),
 frozenset({'Perda Olfato', 'Perda paladar'})]

In [26]:
px.histogram(y=[y for x in agg for y in x]).update_layout(yaxis_title='Sintomas', xaxis_title='Quantidade').update_yaxes(categoryorder='total ascending')

In [27]:
px.histogram(df, x='agente_saude', histnorm='percent').update_layout(yaxis_title='Porcentagem', xaxis_title='Agente de Saude?')

Adicionando mais uma coluna no primeiro DataFrame, neste contendo os dias da semana no qual o contagio foi notificado.

In [28]:
import calendar

days = []

for d in df['data']:
    days.append(calendar.day_name[d.weekday()])

df['week day'] = days

df['week day'] = df['week day'].map({'Sunday': 'Domingo', 'Monday': 'Segunda-Feira', 'Tuesday': 'Terça-Feira', 'Wednesday': 'Quarta-Feira', 'Thursday': 'Quinta-Feira', 'Friday': 'Sexta-Feira', 'Saturday': 'Sábado'})

Aqui vemos a distribuição da divulgação dos casos por dia de semana

In [29]:
px.histogram(data_frame=df, x='week day', labels={'week day': 'Dia da semana'}, category_orders={'week day': ['Domingo', 'Segunda-Feira', 'Terça-Feira', 'Quarta-Feira', 'Quinta-Feira', 'Sexta-Feira', 'Sábado']}).update_layout(xaxis_title='Dia da semana', yaxis_title='Quantidade')

In [30]:
from pdf_reports import pug_to_html, write_report

In [31]:
today = datetime.datetime.now()
casos_ativos = total_ativo['Casos Ativos'].iloc[-1]

def get_week():
    return (today - datetime.datetime.strptime('2020-04-19', '%Y-%m-%d')) // datetime.timedelta(7) + 1

def get_weekday():
    return datetime.datetime.weekday(today)

def get_week_serie():
    week = get_week()
    return new_df[new_df.semana == week]

In [32]:
from selenium.webdriver import Chrome, ChromeOptions
from time import sleep
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
def send_report(week):
    options = ChromeOptions()
    options.add_argument('--user-data-dir=chromium/Default')
    browser = Chrome(options=options)

    browser.get('https://web.whatsapp.com')

    sleep(15)

    browser.find_element_by_xpath(f"//span[@title='{os.getenv('NUMBER')}']").click()
    # browser.send_keys('relatorio.pdf')
    # sleep(3)
    # chat = browser.find_element_by_xpath("//div[@contenteditable='true' and @spellcheck='true']")
    browser.find_element_by_xpath("//span[@data-icon='clip']").click()
    sleep(1)
    browser.find_element_by_xpath("//input[@type='file']").send_keys('{}/relatorio_{}.pdf'.format(os.getcwd(), week))
    sleep(5)
    browser.find_element_by_xpath("//span[@data-icon='send']").click()
    sleep(30)
    browser.quit()

In [33]:
if calendar.day_name[get_weekday()] == 'Saturday':
    html = pug_to_html('modelo_de_relatorio.pug',
    hoje=today,
    df=new_df,
    casos_medio=round(get_week_serie().casos.describe()['mean'], 2),
    recuperados_medio=round(get_week_serie().recuperados.describe()['mean'], 2),
    obitos_medio=round(get_week_serie().mortes.describe()['mean'], 2),
    total_casos_semana=get_week_serie().casos.sum(),
    total_recuperados_semana=get_week_serie().recuperados.sum(),
    total_mortes_semana=get_week_serie().mortes.sum(),
    casos_ativos=casos_ativos,
    total_ativo=total_ativo,
    total_casos=df.shape[0],
    semana=get_week(),
    last_week=new_df[new_df.semana == get_week()],
    )
    write_report(html, 'relatorio_{}.pdf'.format(get_week()))
    send_report(get_week())
    os.remove('relatorio_{}.pdf'.format(get_week()))